In [1]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/P1015.3. Analyses/P1015.3.2. Data Processing/Output'
cur_date = '032925'

library(missForest)
library(readxl)
library(openxlsx)
library(tidyverse)
library(imputeLCMD)
library(factoextra)

# reading in files
ws_df = data.frame(Data = 'WS', read_excel("Input/Woodsmoke_Data_012825.xlsx", sheet = 2)) %>%
    select(-Sample_Number)
wf_df = data.frame(Data = 'WF', read_excel("Input/Wildfire_Data_012825.xlsx", sheet = 2)) %>%
    select(-Sample_Number)
control_df = data.frame(read_excel("Input/Control_Data_032825.xlsx", sheet = 2))  %>%
    select(-Sample_Number)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
an

NOTES TO SELF:
- change 'replicate' to 'instance'

In [2]:
head(ws_df)
head(wf_df)
head(control_df)

,Data,HAWC_ID,Study,Replicate,Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,1263480,Burnet et al. 1990,NA,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


,Data,HAWC_ID,Study,Replicate,Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WF,1289821,Liang et al. 2021,1,PAH,Volume,NA,1-(10-methylanthracen-9-yl)ethanone,5
2,WF,1289821,Liang et al. 2021,1,PAH,Volume,DTXSID50176885,1-Acenaphthenone,1
3,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
4,WF,1289739,Campos et al. 2019,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
5,WF,1289739,Campos et al. 2019,NA,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
6,WF,1289739,Campos et al. 2019,NA,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA


,Data,HAWC_ID,Study,Replicate,Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,TRAP,1298357,Lee at al. 1995,1,PAH,Weight,DTXSID3021774,Acenaphthene,71.17
2,TRAP,1298357,Lee at al. 1995,1,PAH,Weight,DTXSID3023845,Acenaphthylene,19.63
3,TRAP,1298357,Lee at al. 1995,1,PAH,Weight,DTXSID5023902,Benzo(a)anthracene,333.74
4,TRAP,1298357,Lee at al. 1995,1,PAH,Weight,DTXSID2020139,Benzo(a)pyrene,1526.38
5,TRAP,1298357,Lee at al. 1995,1,PAH,Weight,DTXSID80175664,Benzo[b]chrysene,988.96
6,TRAP,1298357,Lee at al. 1995,1,PAH,Weight,DTXSID0023907,Benzo(b)fluoranthene,1334.97


In [3]:
dim(ws_df)
dim(wf_df)
dim(control_df)

[1] 6607    9

[1] 4340    9

[1] 1244    9

# Filter #1

Removing duplicate records for instances that weren't measured at all (MAR). 

In [4]:
`%notin%` <- Negate(`%in%`)

ws_df = ws_df %>%
    filter(Replicate %notin% NA)
wf_df = wf_df %>%
    filter(Replicate %notin% NA)
control_df = control_df %>%
    filter(Replicate %notin% NA)

In [5]:
dim(ws_df)
dim(wf_df)
dim(control_df)

[1] 4203    9

[1] 2378    9

[1] 1215    9

In [6]:
# combining dfs
combined_df = rbind(ws_df, wf_df, control_df)
head(combined_df)

,Data,HAWC_ID,Study,Replicate,Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,267091,Forchhammer et al. 2012,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


In [7]:
length(unique(combined_df$Name))
length(unique(ws_df$Name))
length(unique(wf_df$Name))
length(unique(control_df$Name))

[1] 228

[1] 137

[1] 138

[1] 127

228 unique metals/chemicals total.

# Filter #2: Chemical/Metal Background Filters

Determining what variables have missing data for all variables delineated based on the dfs the data will be split into and analyzed. 

A variable (ie. `Name` which represents a chemical or metal) will be retained if >= 50% of its experimental data points were measurable values. In other words, chemicals were excluded if they were NAs (not evaluated) or had non-detect values across < 50% of the samples.

In [8]:
sample_type_presence_df = combined_df %>%
    # if an experimental value is there count it as being "present"
    mutate(count = ifelse(Value %notin% c('NA', 'ND'), 1, 0)) %>%
    # determining which have at least 50% experimental values for each chemical/metal
    group_by(Metric, Class, Data, Name) %>%
    # calculating the percentage of experimental data points
    reframe(Variable_Presence_Percentage = (sum(count)/n()) * 100) %>%
    filter(Variable_Presence_Percentage >= 50) 

head(sample_type_presence_df)

Metric,Class,Data,Name,Variable_Presence_Percentage
<chr>,<chr>,<chr>,<chr>,<dbl>
VOC,Metal,Diesel,"1,2,4-Trimethylbenzene",100
VOC,Metal,Diesel,1-Hexene,100
VOC,Metal,Diesel,2-Butanone,100
VOC,Metal,Diesel,Benzene,100
VOC,Metal,Diesel,Butanal,100
VOC,Metal,Diesel,Ethylbenzene,100


In [9]:
# filtering the original df
filter1_df = inner_join(sample_type_presence_df[,1:4], combined_df) %>%
    # metals don't have a DTXSID, so making that col their name
    mutate(DTXSID = ifelse(DTXSID != 'NA', DTXSID, Name)) 

head(filter1_df)

Joining with `by = join_by(Metric, Class, Data, Name)`


Metric,Class,Data,Name,HAWC_ID,Study,Replicate,DTXSID,Value
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
VOC,Metal,Diesel,"1,2,4-Trimethylbenzene",1298372,Smits et al. 2012,1,DTXSID6021402,2269000
VOC,Metal,Diesel,"1,2,4-Trimethylbenzene",1298353,Huang et al. 2020,1,DTXSID6021402,70000
VOC,Metal,Diesel,1-Hexene,1298372,Smits et al. 2012,1,DTXSID4025402,3904000
VOC,Metal,Diesel,1-Hexene,1298353,Huang et al. 2020,1,DTXSID4025402,74000
VOC,Metal,Diesel,2-Butanone,1298372,Smits et al. 2012,1,DTXSID3021516,1676000
VOC,Metal,Diesel,2-Butanone,1298353,Huang et al. 2020,1,DTXSID3021516,208000


In [10]:
dim(combined_df)
dim(filter1_df)

[1] 7796    9

[1] 5335    9

Started with 7796 records, 2461 were removed, leaving 5335. 

# Filter 3: Study Background Filter

A study will be retained if >= 50% of its experimental data points were measurable values. In other words, chemicals were excluded if they were NAs (not evaluated) or had non-detect values across < 50% of the samples.

In [11]:
study_type_presence_df = filter1_df %>%
    # if an experimental value is there count it as being "present"
    mutate(count = ifelse(Value %notin% c('NA', 'ND'), 1, 0)) %>%
    # determining which have at least 50% experimental values for each chemical/metal
    group_by(Metric, Class, HAWC_ID) %>%
    # calculating the percentage of experimental data points
    reframe(Study_Presence_Percentage = (sum(count)/n()) * 100) %>%
    filter(Study_Presence_Percentage >= 50) 

head(study_type_presence_df)

Metric,Class,HAWC_ID,Study_Presence_Percentage
<chr>,<chr>,<dbl>,<dbl>
VOC,Metal,1298353,100.00000
VOC,Metal,1298360,100.00000
VOC,Metal,1298372,100.00000
Volume,Metal,2,94.44444
Volume,Metal,266980,100.00000
Volume,Metal,266987,100.00000


In [12]:
filter2_df = inner_join(study_type_presence_df[,1:3], filter1_df)

head(filter2_df)

Joining with `by = join_by(Metric, Class, HAWC_ID)`


Metric,Class,HAWC_ID,Data,Name,Study,Replicate,DTXSID,Value
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
VOC,Metal,1298353,Diesel,"1,2,4-Trimethylbenzene",Huang et al. 2020,1,DTXSID6021402,70000
VOC,Metal,1298353,Diesel,1-Hexene,Huang et al. 2020,1,DTXSID4025402,74000
VOC,Metal,1298353,Diesel,2-Butanone,Huang et al. 2020,1,DTXSID3021516,208000
VOC,Metal,1298353,Diesel,Benzene,Huang et al. 2020,1,DTXSID3039242,114000
VOC,Metal,1298353,Diesel,Ethylbenzene,Huang et al. 2020,1,DTXSID3020596,101000
VOC,Metal,1298353,Diesel,Hexanal,Huang et al. 2020,1,DTXSID2021604,552000


In [13]:
dim(filter1_df)
dim(filter2_df)

[1] 5335    9

[1] 5236    9

299 more records were removed.

# Filter #3: Retaining Metals/or Chemicals that are Present in all Samples

In the intial filter, WS, WF and control samples were combined. However, this time variables (ie. metal or chemical) will be split based on their `Metric` (ie. volume or weight) and then retained if that variable is found in both woodsmoke, wildfire, and control samples within volume or weight samples.

In [14]:
split_filtered_df = filter2_df %>%
    filter(Data %in% c('WS', 'WF')) %>%
    group_by(Data, Metric) %>%
    group_split

split_ws_vol_df = split_filtered_df[[3]]
split_ws_weight_df = split_filtered_df[[4]]
split_wf_vol_df = split_filtered_df[[1]]
split_wf_weight_df = split_filtered_df[[2]]

control_vol_df = filter2_df %>%
    filter(Data %notin% c('WS', 'WF'), Metric == 'Volume')
control_weight_df = filter2_df %>%
    filter(Data %notin% c('WS', 'WF'), Metric == 'Weight')

In [15]:
# seeing how many unique variables are in each df and if they're consistent in each file
length(unique(split_ws_vol_df$Name))
length(unique(split_wf_vol_df$Name))
length(unique(split_ws_weight_df$Name))
length(unique(split_wf_weight_df$Name))
length(unique(control_vol_df$Name))
length(unique(control_weight_df$Name))

[1] 95

[1] 68

[1] 66

[1] 70

[1] 76

[1] 79

In [16]:
# they're not so first getting variables that are in weight or volume samples
# creating vector with common the names between the data types
# common_vol_names = intersect(intersect(unique(split_ws_vol_df$Name), unique(control_vol_df$Name)),consistent_wf_vol_df$Name)
# common_weight_names = intersect(intersect(unique(split_ws_weight_df$Name), unique(control_weight_df$Name)),consistent_wf_weight_df$Name)

# consistent_wf_vol_df = split_wf_vol_df %>%
#     filter(Value != 'NA', Name %in% common_vol_names)
# consistent_ws_vol_df = split_ws_vol_df %>%
#      filter(Value != 'NA', Name %in% common_vol_names)
# consistent_control_vol_df = control_vol_df %>%
#      filter(Value != 'NA', Name %in% common_vol_names)


# consistent_wf_weight_df = split_wf_weight_df %>%
#      filter(Value != 'NA', Name %in% common_weight_names)
# consistent_ws_weight_df = split_ws_weight_df %>%
#      filter(Value != 'NA', Name %in% common_weight_names)
# consistent_control_weight_df = control_weight_df %>%
#      filter(Value != 'NA', Name %in% common_weight_names)

consistent_wf_vol_df = split_wf_vol_df %>%
    filter(Value != 'NA', Name %in% unique(split_ws_vol_df$Name))
consistent_ws_vol_df = split_ws_vol_df %>%
    filter(Value != 'NA', Name %in% consistent_wf_vol_df$Name)
consistent_wf_weight_df = split_wf_weight_df %>%
    filter(Value != 'NA', Name %in% unique(split_ws_weight_df$Name))
consistent_ws_weight_df = split_ws_weight_df %>%
    filter(Value != 'NA', Name %in% consistent_wf_weight_df$Name)

# double checking
length(unique(consistent_wf_vol_df$Name))
length(unique(consistent_ws_vol_df$Name))
length(unique(control_vol_df$Name))
#length(unique(consistent_control_vol_df$Name))
length(unique(consistent_wf_weight_df$Name))
length(unique(consistent_ws_weight_df$Name))
length(unique(control_weight_df$Name))
#length(unique(consistent_control_weight_df$Name))

[1] 32

[1] 32

[1] 76

[1] 23

[1] 23

[1] 79

There were 98, 68, 94 and 70 woodsmoke weight, woodsmoke volume, wildfire weight, and wildfire volume samples, respectively. 33 variables were common between volume samples and 34 were common between weight records and will be retained. REWORD!!

In [17]:
# recombining data
vol_df = rbind(consistent_ws_vol_df, consistent_wf_vol_df, control_vol_df)#consistent_control_vol_df)
weight_df = rbind(consistent_ws_weight_df, consistent_wf_weight_df, control_weight_df)#consistent_control_weight_df)

head(vol_df)

Metric,Class,HAWC_ID,Data,Name,Study,Replicate,DTXSID,Value
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
Volume,Metal,266980,WS,K,Stockfelt et al. 2012,1,K,9700
Volume,Metal,266980,WS,K,Stockfelt et al. 2012,2,K,8800
Volume,Metal,266980,WS,Rb,Stockfelt et al. 2012,1,Rb,73
Volume,Metal,266980,WS,Rb,Stockfelt et al. 2012,2,Rb,105
Volume,Metal,266987,WS,Ca,Seagrave 2005,1,Ca,49200
Volume,Metal,266987,WS,Cd,Seagrave 2005,1,Cd,3500


# Square Root Imputation

Taking the square root of the mean to impute MNAR data.

In [18]:
# first determining number of ND and NA values in each df
vol_df %>%
    filter(Value %in% c('NA', 'ND')) %>%
    group_by(Value) %>%
    summarize(n())

weight_df %>%
    filter(Value %in% c('NA', 'ND')) %>%
    group_by(Value) %>%
    summarize(n())

Value,n()
<chr>,<int>
NA,8
ND,25


Value,n()
<chr>,<int>
NA,15
ND,28


In [19]:
dim(vol_df)
dim(weight_df)

[1] 1378    9

[1] 1474    9

REWORD!! Only the weight dataframe had non-detect values representing MNAR data. The remaining MAR data from the weight df will be imputed using random forest (RF).

In [20]:
# creating a df for MAR data
mar_weight_df = weight_df %>%
    filter(Value == 'NA')
mar_vol_df = vol_df %>%
    filter(Value == 'NA')

preimputed_weight_df = anti_join(weight_df, mar_weight_df) 
preimputed_vol_df = anti_join(vol_df, mar_vol_df) 

preimputed_weight_df$Value = as.numeric(preimputed_weight_df$Value)
preimputed_vol_df$Value = as.numeric(preimputed_vol_df$Value)
mar_weight_df$Value = as.numeric(mar_weight_df$Value)
mar_vol_df$Value = as.numeric(mar_vol_df$Value)

# reordering cols
preimputed_weight_df = preimputed_weight_df[,c(3,1,4,6,7,2,8,5,9)]
preimputed_vol_df = preimputed_vol_df[,c(3,1,4,6,7,2,8,5,9)]

head(preimputed_weight_df)

Joining with `by = join_by(Metric, Class, HAWC_ID, Data, Name, Study,
Replicate, DTXSID, Value)`
Joining with `by = join_by(Metric, Class, HAWC_ID, Data, Name, Study,
Replicate, DTXSID, Value)`
Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”


HAWC_ID,Metric,Data,Study,Replicate,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cr,Cr,21.4
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cr,Cr,21.3
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cu,Cu,65.6
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cu,Cu,34.4
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Fe,Fe,1323.0
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Fe,Fe,1773.0


In [21]:
# creating a df for MNAR data
mnar_weight_df = preimputed_weight_df %>%
    filter(is.na(Value))
mnar_vol_df = preimputed_vol_df %>%
    filter(is.na(Value))

head(mnar_weight_df)

HAWC_ID,Metric,Data,Study,Replicate,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Mn,Mn,NA
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Mn,Mn,NA
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Ca,Ca,NA
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Fe,Fe,NA
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Mn,Mn,NA
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Mn,Mn,NA


In [22]:
square_root_imputation = function(mnar_df, preimputed_df){
    # ADD WORDS

    id_df = mnar_df[is.na(mnar_df$Value),]
    study_ids = unique(id_df$HAWC_ID)
    
    imputed_df = data.frame()
    for (i in 1:length(id_df$HAWC_ID)){
    
        filtered_df = preimputed_df %>%
            filter(HAWC_ID == id_df$HAWC_ID[i], DTXSID == id_df$DTXSID[i])
    
        na_df = filtered_df %>%
            filter(is.na(Value)) %>%
            mutate(Value = sqrt(mean(filtered_df$Value, na.rm = TRUE)))
    
        # combining rows of data
        combined_df = rbind(filtered_df %>%
            filter(!is.na(Value)), na_df)
    
        imputed_df = rbind(imputed_df, combined_df) %>%
                        unique()
    }
    return(imputed_df)    
    
}

# calling fn
imputed_vol_df = square_root_imputation(mnar_vol_df, preimputed_vol_df)
imputed_weight_df = square_root_imputation(mnar_weight_df, preimputed_weight_df)
    
head(imputed_weight_df)

HAWC_ID,Metric,Data,Study,Replicate,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Mn,Mn,NaN
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Mn,Mn,NaN
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Ca,Ca,7120.00000
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Ca,Ca,84.38009
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Fe,Fe,1172.00000
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Fe,Fe,34.23449


In [23]:
# there are still a few variables that didn't have any values to do the square root imputation
# their imputed values will be based off of distributions from other studies
other_studies_vol_df = semi_join(preimputed_vol_df %>%
                filter(!is.na(Value)), imputed_vol_df %>%
                               filter(is.na(Value)) %>%
    select(2,3,6,7)) %>%
    group_by(DTXSID) %>%
    summarize(Value = sqrt(mean(Value)))

other_studies_weight_df = semi_join(preimputed_weight_df %>%
                filter(!is.na(Value)), imputed_weight_df %>%
                               filter(is.na(Value)) %>%
    select(2,3,6,7)) %>%
    group_by(DTXSID) %>%
    summarize(Value = sqrt(mean(Value)))

head(other_studies_vol_df)
head(other_studies_weight_df)

Joining with `by = join_by(Metric, Data, Class, DTXSID)`
Joining with `by = join_by(Metric, Data, Class, DTXSID)`


DTXSID,Value
<chr>,<dbl>
DTXSID2020139,239.930851
DTXSID3021774,1.467708
DTXSID5023908,300.832966


DTXSID,Value
<chr>,<dbl>
Mn,23.15913
Ni,19.22008


In [24]:
# removing NAs from the imputed df 
removed_imputed_vol_df = imputed_vol_df %>%
    filter(!is.na(Value))
removed_imputed_weight_df = imputed_weight_df %>%
    filter(!is.na(Value))

# adding in the newly imputed values back in 
all_imputed_vol_df = cbind(imputed_vol_df %>%
    filter(is.na(Value)) %>%
    select(-Value), Value = c(rep(other_studies_vol_df$Value[3],2), rep(other_studies_vol_df$Value[1],2),
                     rep(other_studies_vol_df$Value[2],2)))
all_imputed_vol_df = rbind(removed_imputed_vol_df, all_imputed_vol_df)

all_imputed_weight_df = cbind(imputed_weight_df %>%
    filter(is.na(Value)) %>%
    select(-Value), Value = c(rep(other_studies_weight_df$Value[1],2), rep(other_studies_weight_df$Value[2],2),
                     rep(other_studies_weight_df$Value[1],2)))
all_imputed_weight_df = rbind(removed_imputed_vol_df, all_imputed_weight_df)
# left_join(imputed_df %>%
#         filter(is.na(Value)) %>%
#         select(-Value), other_studies_df)

head(all_imputed_weight_df)

HAWC_ID,Metric,Data,Study,Replicate,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1298351,Volume,TRAP,Gunawardena et al. 2012,3,Metal,Cr,Cr,20
1298351,Volume,TRAP,Gunawardena et al. 2012,4,Metal,Cr,Cr,30
1298351,Volume,TRAP,Gunawardena et al. 2012,6,Metal,Cr,Cr,30
1298351,Volume,TRAP,Gunawardena et al. 2012,7,Metal,Cr,Cr,90
1298351,Volume,TRAP,Gunawardena et al. 2012,10,Metal,Cr,Cr,80
1298351,Volume,TRAP,Gunawardena et al. 2012,11,Metal,Cr,Cr,20


In [25]:
# creating 1 df
mnar_imputed_vol_df = rbind(#imputed data
                               all_imputed_vol_df,
                               # adding in MAR data
                               mar_vol_df,
                               # adding back in non-imputed data
                               anti_join(preimputed_vol_df %>%
                                         filter(HAWC_ID %notin% all_imputed_vol_df$HAWC_ID), mnar_vol_df))
mnar_imputed_weight_df = rbind(# imputed data
                               all_imputed_weight_df,
                               # adding in MAR data
                               mar_weight_df,
                               # adding back in non-imputed data
                               anti_join(preimputed_weight_df %>%
                                         filter(HAWC_ID %notin% all_imputed_weight_df$HAWC_ID), mnar_weight_df))

head(mnar_imputed_weight_df)
dim(mnar_imputed_vol_df)
dim(mnar_imputed_weight_df)

Joining with `by = join_by(HAWC_ID, Metric, Data, Study, Replicate, Class,
DTXSID, Name, Value)`
Joining with `by = join_by(HAWC_ID, Metric, Data, Study, Replicate, Class,
DTXSID, Name, Value)`


HAWC_ID,Metric,Data,Study,Replicate,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1298351,Volume,TRAP,Gunawardena et al. 2012,3,Metal,Cr,Cr,20
1298351,Volume,TRAP,Gunawardena et al. 2012,4,Metal,Cr,Cr,30
1298351,Volume,TRAP,Gunawardena et al. 2012,6,Metal,Cr,Cr,30
1298351,Volume,TRAP,Gunawardena et al. 2012,7,Metal,Cr,Cr,90
1298351,Volume,TRAP,Gunawardena et al. 2012,10,Metal,Cr,Cr,80
1298351,Volume,TRAP,Gunawardena et al. 2012,11,Metal,Cr,Cr,20


[1] 1166    9

[1] 1475    9

In [26]:
# dim(vol_df)
# dim(mar_vol_df)
# dim(mnar_vol_df)
# 1378 - 8 - 25
# dim(anti_join(preimputed_vol_df, mnar_vol_df)) 

# dim(all_imputed_vol_df) # yes!
# dim(imputed_vol_df)
# dim(preimputed_vol_df %>%
#                   filter(HAWC_ID %notin% all_imputed_vol_df$HAWC_ID))
# 1095 + 63 

# Filter #4: Double Checking MAR Records

Making sure that when the data is split based on `Class` and transformed into a wide format, we have at least 50% of the records with numeric values for each metal or chemical.

In [27]:
split_vol_df = mnar_imputed_vol_df %>%
    # averaging to get rid of replicates
    group_by(HAWC_ID, Name) %>%
    mutate(Avg_Value = mean(Value)) %>%
    ungroup() %>%
    select(-c("Replicate", "Value", "Name")) %>%
    unique() %>%
    group_by(Class) %>%
    group_split

metal_vol_df = split_vol_df[[1]]
pah_vol_df = split_vol_df[[2]]
voc_vol_df = split_vol_df[[3]]

head(metal_vol_df)

HAWC_ID,Metric,Data,Study,Class,DTXSID,Avg_Value
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1298351,Volume,TRAP,Gunawardena et al. 2012,Metal,Cr,NA
1298351,Volume,TRAP,Gunawardena et al. 2012,Metal,Cu,NA
2,Volume,TRAP,USEPA,Metal,Cu,NA
266980,Volume,WS,Stockfelt et al. 2012,Metal,K,9250
266980,Volume,WS,Stockfelt et al. 2012,Metal,Rb,89
266987,Volume,WS,Seagrave 2005,Metal,Ca,49200


In [28]:
split_weight_df = mnar_imputed_weight_df %>%
    # averaging to get rid of replicates
    group_by(HAWC_ID, Name) %>%
    mutate(Avg_Value = mean(Value)) %>%
    ungroup() %>%
    select(-c("Replicate", "Value", "Name")) %>%
    unique() %>%
    group_by(Class) %>%
    group_split

metal_weight_df = split_weight_df[[1]]
pah_weight_df = split_weight_df[[2]]

head(metal_weight_df)

HAWC_ID,Metric,Data,Study,Class,DTXSID,Avg_Value
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1298351,Volume,TRAP,Gunawardena et al. 2012,Metal,Cr,29.68328
1298351,Volume,TRAP,Gunawardena et al. 2012,Metal,Cu,13.41043
267105,Weight,WS,Danielsen et al. 2011,Metal,Mn,23.15913
1040882,Weight,WS,Nordin et al. 2015,Metal,Mn,19.22008
1040882,Weight,WS,Nordin et al. 2015,Metal,Ni,23.15913
1298342,Weight,Diesel,Bendtsen et al. 2020,Metal,Ca,NA


Converting values to 0s and 1s to count number of experimental and NDs previously imputed. Chemicals/metals that don't have at least 50% of values present will be removed.

In [29]:
wider_metal_vol_df = metal_vol_df %>%
    # creating sample ids
    unite("SampleID", c(colnames(metal_vol_df)[c(1,3,5)]), remove = FALSE) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value") 
wider_pah_vol_df = pah_vol_df %>%
    # creating sample ids
    unite("SampleID", c(colnames(pah_vol_df)[c(1,3,5)]), remove = FALSE) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value")
wider_voc_vol_df = voc_vol_df %>%
    # creating sample ids
    unite("SampleID", c(colnames(pah_vol_df)[c(1,3,5)]), remove = FALSE) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value")

wider_metal_weight_df = metal_weight_df %>%
    unite("SampleID", c(colnames(metal_weight_df)[c(1,3,5)]), remove = FALSE) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value") 
wider_pah_weight_df = pah_weight_df %>%
    unite("SampleID", c(colnames(pah_weight_df)[c(1,3,5)]), remove = FALSE) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value")  

head(wider_metal_weight_df)

SampleID,HAWC_ID,Metric,Data,Study,Class,Cr,Cu,Mn,Ni,⋯,B,Ga,Rb,Mo,Sn,As,Pb,Sb,Sr,V
<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1298351_TRAP_Metal,1298351,Volume,TRAP,Gunawardena et al. 2012,Metal,29.68328,13.41043,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
267105_WS_Metal,267105,Weight,WS,Danielsen et al. 2011,Metal,NA,NA,23.15913,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1040882_WS_Metal,1040882,Weight,WS,Nordin et al. 2015,Metal,NA,NA,19.22008,23.15913,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1298342_Diesel_Metal,1298342,Weight,Diesel,Bendtsen et al. 2020,Metal,22.33333,4597.11633,72.50000,46.33333,⋯,NA,1,1.333333,NA,NA,NA,NA,NA,NA,NA
1298343_Diesel_Metal,1298343,Weight,Diesel,Bhavaraju et al. 2014,Metal,2.00000,33.00000,NA,3.00000,⋯,3.5,NA,NA,3.5,NA,NA,NA,NA,NA,NA
1298344_Diesel_Metal,1298344,Weight,Diesel,Coufalik et al. 2019,Metal,0.01070,0.30000,0.01960,0.01390,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [30]:
data_percentage = function(wider_dataset){
    # ADD WORDS!!
    
    # converting values to 0s and 1s
    summed_df = wider_dataset %>%
        mutate(across(7:ncol(wider_dataset), ~ifelse(is.na(.x), 0, 1)))

    # identifying metals/chemicals w/ < 50% of experimental data
    var_percentage_df = data.frame(Percent = colSums(summed_df[,7:ncol(summed_df)])/nrow(summed_df) * 100) %>%
        filter(Percent <= 50)
    # identifying studies w/ < 10% of experimental data
    study_percentage_df = data.frame(HAWC_ID = wider_dataset$HAWC_ID,
                                     Percent = rowSums(summed_df[,7:ncol(summed_df)])/ncol(summed_df) * 100) %>%
        filter(Percent <= 10)

    return(list(var_percentage_df, study_percentage_df))
}

# calling fn
percentage_metal_vol_df = data_percentage(wider_metal_vol_df)
percentage_pah_vol_df = data_percentage(wider_pah_vol_df) 
percentage_voc_vol_df = data_percentage(wider_voc_vol_df) 
percentage_metal_weight_df = data_percentage(wider_metal_weight_df) 
percentage_pah_weight_df = data_percentage(wider_pah_weight_df)

head(percentage_metal_weight_df[[1]])

,Percent
,<dbl>
Mg,42.222222
Co,8.888889
Ba,11.111111
K,42.222222
Al,48.888889
B,8.888889


In [31]:
# removing those metals/chemicals & studies from the OG df
preimputed_metal_vol_df = wider_metal_vol_df[,colnames(wider_metal_vol_df) %notin% rownames(percentage_metal_vol_df[[1]])]
preimputed_pah_vol_df = wider_pah_vol_df[,colnames(wider_pah_vol_df) %notin% rownames(percentage_pah_vol_df[[1]])]
preimputed_voc_vol_df = wider_voc_vol_df[,colnames(wider_voc_vol_df) %notin% rownames(percentage_voc_vol_df[[1]])]
preimputed_metal_weight_df = wider_metal_weight_df[,colnames(wider_metal_weight_df) %notin% rownames(percentage_metal_weight_df[[1]])]
preimputed_pah_weight_df = wider_pah_weight_df[,colnames(wider_pah_weight_df) %notin% rownames(percentage_pah_weight_df[[1]])]

preimputed_metal_vol_df = preimputed_metal_vol_df %>% filter(HAWC_ID %notin% percentage_metal_vol_df[[2]]$HAWC_ID)
preimputed_pah_vol_df = preimputed_pah_vol_df %>% filter(HAWC_ID %notin% percentage_pah_vol_df[[2]]$HAWC_ID)
preimputed_metal_vol_df = preimputed_metal_vol_df %>% filter(HAWC_ID %notin% percentage_metal_vol_df[[2]]$HAWC_ID)
preimputed_metal_weight_df = preimputed_metal_weight_df %>% filter(HAWC_ID %notin% percentage_metal_weight_df[[2]]$HAWC_ID)
preimputed_pah_weight_df = preimputed_pah_weight_df %>% filter(HAWC_ID %notin% percentage_pah_weight_df[[2]]$HAWC_ID)

head(preimputed_metal_weight_df)

SampleID,HAWC_ID,Metric,Data,Study,Class,Cr,Cu,Mn,Ni,Ca,Fe,Zn
<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1298342_Diesel_Metal,1298342,Weight,Diesel,Bendtsen et al. 2020,Metal,22.33333,4597.116,72.5000,46.33333,NA,824.00,NA
1298343_Diesel_Metal,1298343,Weight,Diesel,Bhavaraju et al. 2014,Metal,2.00000,33.000,NA,3.00000,367,90.00,92.50
1298344_Diesel_Metal,1298344,Weight,Diesel,Coufalik et al. 2019,Metal,0.01070,0.300,0.0196,0.01390,NA,0.55,0.32
1298349_Diesel_Metal,1298349,Weight,Diesel,Fox et al. 2015,Metal,NA,1155.000,NA,NA,475,250.00,780.00
1298372_Diesel_Metal,1298372,Weight,Diesel,Smits et al. 2012,Metal,20.00000,210.000,73.0000,NA,311,161.00,659.00
271571_WS_Metal,271571,Weight,WS,Dilger et al. 2016,Metal,3.40000,4.100,2138.0000,0.50000,NA,114917.40,4178.80


In [32]:
# How many metals/chemicals are now left in the dataset?
ncol(preimputed_metal_vol_df) - 6
ncol(preimputed_pah_vol_df) - 6
ncol(preimputed_voc_vol_df) - 6
ncol(preimputed_metal_weight_df) - 6
ncol(preimputed_pah_weight_df) - 6

[1] 6

[1] 4

[1] 3

[1] 7

[1] 2

ADD WORDS. 

# RF Imputation

In [33]:
random_forest_imputation = function(dataset){
    # ADD WORDS
    
    preimputed_df = dataset[,7:ncol(dataset)]
    
    imputed_RF_object = missForest(as.matrix(preimputed_df))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(dataset[,1:6], imputed_RF_df)) %>%
        pivot_longer(cols = 7:ncol(dataset), names_to = "DTXSID", values_to = "Value")
    
    return(imputed_final_df)
}

# calling fn
rf_metal_vol_imputed_df = random_forest_imputation(preimputed_metal_vol_df) 
rf_pah_vol_imputed_df = random_forest_imputation(preimputed_pah_vol_df)
rf_voc_vol_imputed_df = random_forest_imputation(preimputed_voc_vol_df)
rf_metal_weight_imputed_df = random_forest_imputation(preimputed_metal_weight_df) 
#rf_pah_weight_imputed_df = random_forest_imputation(preimputed_pah_weight_df)

rf_imputed_df = rbind(rf_metal_vol_imputed_df, rf_pah_vol_imputed_df, 
                      rf_voc_vol_imputed_df, rf_metal_weight_imputed_df#,
                      #rf_pah_weight_imputed_df
                     ) %>%
    mutate(Name = DTXSID) 

rf_imputed_df = rf_imputed_df[,c(3,6,2,4,7,9,8)]

head(rf_imputed_df)

Metric,Class,HAWC_ID,Data,DTXSID,Name,Value
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
Volume,Metal,2,TRAP,K,K,35903.27509
Volume,Metal,2,TRAP,Ca,Ca,682.75550
Volume,Metal,2,TRAP,Cd,Cd,0.09450
Volume,Metal,2,TRAP,Fe,Fe,231.21833
Volume,Metal,2,TRAP,Mn,Mn,2.17625
Volume,Metal,2,TRAP,Ni,Ni,0.73025


In [34]:
# combining data
final_df = rbind(rf_metal_vol_imputed_df, rf_pah_vol_imputed_df, 
                      rf_voc_vol_imputed_df, rf_metal_weight_imputed_df#,
                      #rf_pah_weight_imputed_df
                     ) %>%
    mutate(Name = DTXSID) 

final_df = final_df[,c(2:7,9,8)]

head(final_df)
dim(final_df)

HAWC_ID,Metric,Data,Study,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
2,Volume,TRAP,USEPA,Metal,K,K,35903.27509
2,Volume,TRAP,USEPA,Metal,Ca,Ca,682.75550
2,Volume,TRAP,USEPA,Metal,Cd,Cd,0.09450
2,Volume,TRAP,USEPA,Metal,Fe,Fe,231.21833
2,Volume,TRAP,USEPA,Metal,Mn,Mn,2.17625
2,Volume,TRAP,USEPA,Metal,Ni,Ni,0.73025


[1] 511   8

In [35]:
# exporting
write.xlsx(final_df, paste0(Output,"/", "Processed_Data_", cur_date, ".xlsx"), 
           rowNames = FALSE)